## Reajustar modelo, devido inversão na variável PAY_AMT3

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Curso_Logistica/dataset_trat.csv',sep=';')
df.head()

,ID,SAFRA,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,200510,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,200510,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,200510,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,200510,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,200510,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


## Transformações

In [4]:
df['SUM_BILL'] = df['BILL_AMT1'] + df['BILL_AMT2'] + df['BILL_AMT3'] + df['BILL_AMT4'] + df['BILL_AMT5'] + df['BILL_AMT6']   
df['AVG_BILL'] = df['SUM_BILL'] / 6
df['max_BILL'] = df[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].max(axis=1)

df['SUM_PAY'] = df['PAY_AMT1'] + df['PAY_AMT2'] + df['PAY_AMT3'] + df['PAY_AMT4'] + df['PAY_AMT5'] + df['PAY_AMT6']
df['AVG_PAY'] = df['SUM_PAY'] / 6
df['max_PAY'] = df[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5','PAY_AMT6']].max(axis=1)

## Divisão do conjunto de dados

In [5]:
OOT = df.loc[df['SAFRA'] >= 200605 ]
OOT['SAFRA'].groupby(OOT['SAFRA']).count()

SAFRA
200605    3000
200606    3000
200607    3000
Name: SAFRA, dtype: int64

In [6]:
DEV = df.loc[df['SAFRA'] < 200605 ]
DEV['SAFRA'].groupby(DEV['SAFRA']).count()

SAFRA
200510    3000
200511    3000
200512    3000
200601    3000
200602    3000
200603    3000
200604    3000
Name: SAFRA, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(DEV.drop(columns=['default']), DEV['default'] ,test_size=0.3, random_state=123)

In [8]:
X_train['SAFRA'].groupby(X_train['SAFRA']).count()

SAFRA
200510    2103
200511    2124
200512    2110
200601    2081
200602    2075
200603    2102
200604    2105
Name: SAFRA, dtype: int64

In [9]:
X_test['SAFRA'].groupby(X_test['SAFRA']).count()

SAFRA
200510    897
200511    876
200512    890
200601    919
200602    925
200603    898
200604    895
Name: SAFRA, dtype: int64

In [10]:
X_train.loc[X_train['BILL_AMT1'] < 0, 'BILL_AMT1'] = -1
X_train.loc[X_train['BILL_AMT2'] < 0, 'BILL_AMT2'] = -1
X_train.loc[X_train['BILL_AMT3'] < 0, 'BILL_AMT3'] = -1
X_train.loc[X_train['BILL_AMT4'] < 0, 'BILL_AMT4'] = -1
X_train.loc[X_train['BILL_AMT5'] < 0, 'BILL_AMT5'] = -1
X_train.loc[X_train['BILL_AMT6'] < 0, 'BILL_AMT6'] = -1


In [11]:
X_train.loc[X_train['PAY_AMT1'] > 18006.25 , 'PAY_AMT1'] = 18006.25
X_train.loc[X_train['PAY_AMT2'] > 18987.5  , 'PAY_AMT2'] = 18987.5
X_train.loc[X_train['PAY_AMT3'] > 	15500.7 , 'PAY_AMT3'] = 	15500.7
X_train.loc[X_train['PAY_AMT4'] > 	15150.9 , 'PAY_AMT4'] = 	15150.9
X_train.loc[X_train['PAY_AMT5'] > 15620.05 , 'PAY_AMT5'] = 15620.05
X_train.loc[X_train['PAY_AMT6'] > 16992.4 , 'PAY_AMT6'] = 16992.4


In [12]:
X_train_trat = X_train.copy()

In [13]:
X_train_trat.head()

,ID,SAFRA,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SUM_BILL,AVG_BILL,max_BILL,SUM_PAY,AVG_PAY,max_PAY
10910,10911,200601,220000,2,2,1,53,0,0,0,0,0,0,207592,212099,214669,173187,176496,169649,8165.0,6889.0,6033.0,6247.0,6142.0,6146.0,1153692,192282.000000,214669,39622,6603.666667,8165
12563,12564,200602,300000,2,1,2,31,0,0,0,-1,-1,-1,11158,1616,3658,658,662,0,1000.0,2700.0,658.0,662.0,0.0,0.0,17752,2958.666667,11158,5020,836.666667,2700
369,370,200510,20000,2,2,1,32,3,2,0,0,0,0,20631,20010,19703,11084,10918,10985,0.0,1168.0,384.0,390.0,632.0,616.0,93331,15555.166667,20631,3190,531.666667,1168
7702,7703,200512,410000,1,3,2,38,-1,-1,-1,-1,-1,-1,4822,232,2255,2546,2650,6286,233.0,2266.0,2558.0,2663.0,6303.0,12.0,18791,3131.833333,6286,14035,2339.166667,6303
7464,7465,200512,100000,2,2,2,25,0,0,0,-1,-1,-1,8791,4910,0,674,0,980,1000.0,0.0,674.0,0.0,980.0,4413.0,15355,2559.166667,8791,7067,1177.833333,4413


## Avaliação da relevância e redundância dos atributos

In [14]:
## Instalação do optbinning: http://gnpalencia.org/optbinning/installation.html
#! pip install optbinning

In [15]:
from optbinning import OptimalBinning
from optbinning import BinningProcess

## Modelagem

### Trabalhar com variáveis categóricas

In [16]:
import pickle
from optbinning import OptimalBinning
from optbinning import BinningProcess

with open('/content/drive/MyDrive/Curso_Logistica/binning_process.pickle', 'rb') as handle:
  binning_process = pickle.load(handle)

In [17]:
binning_process

BinningProcess(binning_fit_params=None, binning_transform_params=None,
               categorical_variables=['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0',
                                      'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
                                      'PAY_6'],
               max_bin_size=None, max_n_bins=3, max_n_prebins=20,
               max_pvalue=None, max_pvalue_policy='consecutive',
               min_bin_size=None, min_n_bins=None, min_prebin_size=0.1,
               n_jobs=None, selection_criteria...None,
               split_digits=None,
               variable_names=['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE',
                               'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4',
                               'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
                               'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',
                               'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
                               'PAY_AMT4', 'PAY_AMT5', 'PAY_A

In [18]:
X_train_transf = binning_process.transform(X_train_trat,metric='bins')

In [19]:
X_train_transf.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SUM_BILL,AVG_BILL,max_BILL,SUM_PAY,AVG_PAY,max_PAY
0,"[125000.00, inf)",[2],[2],[1 3],"[41.50, inf)",[ 0 -2 -1],[ 8 0 1 -1],[-1 0],[ 0 -2 6],[ 0 -2],[ 0 -2],"[65968.50, inf)","[35166.00, inf)","[32102.00, inf)","[8438.00, inf)","[8150.50, inf)","[5865.00, inf)","[4902.50, inf)","[4966.50, inf)","[2918.50, inf)","[2851.50, inf)","[3997.50, inf)","[3781.00, inf)","[290629.00, inf)","[48438.17, inf)","[35558.00, inf)","[13302.50, 60701.50)","[2217.08, 10116.92)","[4873.50, 24396.00)"
1,"[125000.00, inf)",[2],[0 4 6 5 1],[0 2],"[25.50, 41.50)",[ 0 -2 -1],[ 8 0 1 -1],[-1 0],[-1],[-1],[-1],"[781.50, 65968.50)","[-inf, 11880.50)","[-inf, 12238.00)","[-inf, 2509.00)","[-inf, 1326.50)","[-inf, 399.50)","[21.50, 4902.50)","[91.00, 4966.50)","[1.50, 2918.50)","[11.50, 2851.50)","[-inf, 0.50)","[-inf, 941.50)","[8633.00, 290629.00)","[1438.83, 48438.17)","[2984.50, 35558.00)","[-inf, 13302.50)","[-inf, 2217.08)","[-inf, 4873.50)"
2,"[-inf, 45000.00)",[2],[2],[1 3],"[25.50, 41.50)",[6 5 8 4 2 3 7],[4 2 7 3 5 6],[-1 0],[ 0 -2 6],[ 0 -2],[ 0 -2],"[781.50, 65968.50)","[11880.50, 35166.00)","[12238.00, 32102.00)","[8438.00, inf)","[8150.50, inf)","[5865.00, inf)","[-inf, 21.50)","[91.00, 4966.50)","[1.50, 2918.50)","[11.50, 2851.50)","[0.50, 3997.50)","[-inf, 941.50)","[8633.00, 290629.00)","[1438.83, 48438.17)","[2984.50, 35558.00)","[-inf, 13302.50)","[-inf, 2217.08)","[-inf, 4873.50)"
3,"[125000.00, inf)",[1],[3],[0 2],"[25.50, 41.50)",[ 0 -2 -1],[ 8 0 1 -1],[-1 0],[-1],[-1],[-1],"[781.50, 65968.50)","[-inf, 11880.50)","[-inf, 12238.00)","[2509.00, 8438.00)","[1326.50, 8150.50)","[5865.00, inf)","[21.50, 4902.50)","[91.00, 4966.50)","[1.50, 2918.50)","[11.50, 2851.50)","[3997.50, inf)","[-inf, 941.50)","[8633.00, 290629.00)","[1438.83, 48438.17)","[2984.50, 35558.00)","[13302.50, 60701.50)","[2217.08, 10116.92)","[4873.50, 24396.00)"
4,"[45000.00, 125000.00)",[2],[2],[0 2],"[-inf, 25.50)",[ 0 -2 -1],[ 8 0 1 -1],[-1 0],[-1],[-1],[-1],"[781.50, 65968.50)","[-inf, 11880.50)","[-inf, 12238.00)","[-inf, 2509.00)","[-inf, 1326.50)","[399.50, 5865.00)","[21.50, 4902.50)","[-inf, 91.00)","[1.50, 2918.50)","[-inf, 11.50)","[0.50, 3997.50)","[3781.00, inf)","[8633.00, 290629.00)","[1438.83, 48438.17)","[2984.50, 35558.00)","[-inf, 13302.50)","[-inf, 2217.08)","[-inf, 4873.50)"


In [20]:
y = y_train.reset_index()

In [21]:
y.head()

,index,default
0,10910,1
1,12563,0
2,369,1
3,7702,0
4,7464,0


In [28]:
import statsmodels.formula.api as smf
log_reg = smf.logit(formula = " y['default'] ~ C(LIMIT_BAL, Treatment(reference=0)) + \
                                          C(PAY_0, Treatment(reference=0)) + \
                                          C(PAY_AMT1, Treatment(reference=0)) + \
                                          C(AVG_PAY, Treatment(reference=0))  \
                                     ", data=X_train_transf).fit()

Optimization terminated successfully.
         Current function value: 0.453352
         Iterations 6


In [29]:
print(log_reg.summary2())

                                                Results: Logit
Model:                             Logit                           Pseudo R-squared:                0.159     
Dependent Variable:                y['default']                    AIC:                             13346.5452
Date:                              2021-03-26 18:07                BIC:                             13414.9056
No. Observations:                  14700                           Log-Likelihood:                  -6664.3   
Df Model:                          8                               LL-Null:                         -7923.7   
Df Residuals:                      14691                           LLR p-value:                     0.0000    
Converged:                         1.0000                          Scale:                           1.0000    
No. Iterations:                    6.0000                                                                     
-------------------------------------------------

### Modelo FInal



```



==============================================================================================================
Model:                             Logit                           Pseudo R-squared:                0.159     
Dependent Variable:                y['default']                    AIC:                             13346.5452
Date:                              2021-03-26 12:41                BIC:                             13414.9056
No. Observations:                  14700                           Log-Likelihood:                  -6664.3   
Df Model:                          8                               LL-Null:                         -7923.7   
Df Residuals:                      14691                           LLR p-value:                     0.0000    
Converged:                         1.0000                          Scale:                           1.0000    
No. Iterations:                    6.0000                                                                     
--------------------------------------------------------------------------------------------------------------
                                                               Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                                     -0.9964   0.0741 -13.4536 0.0000 -1.1416 -0.8513
C(LIMIT_BAL, Treatment(reference=0))[T.[125000.00, inf)]      -0.3884   0.0644  -6.0283 0.0000 -0.5147 -0.2621
C(LIMIT_BAL, Treatment(reference=0))[T.[45000.00, 125000.00)] -0.2507   0.0630  -3.9763 0.0001 -0.3743 -0.1271
C(PAY_0, Treatment(reference=0))[T.[1]]                        0.8618   0.0637  13.5341 0.0000  0.7370  0.9866
C(PAY_0, Treatment(reference=0))[T.[6 5 8 4 2 3 7]]            2.4296   0.0621  39.1450 0.0000  2.3080  2.5513
C(PAY_AMT1, Treatment(reference=0))[T.[21.50, 4902.50)]       -0.2906   0.0598  -4.8606 0.0000 -0.4078 -0.1734
C(PAY_AMT1, Treatment(reference=0))[T.[4902.50, inf)]         -0.4409   0.0803  -5.4872 0.0000 -0.5984 -0.2834
C(AVG_PAY, Treatment(reference=0))[T.[10116.92, inf)]         -0.7438   0.0981  -7.5837 0.0000 -0.9360 -0.5516
C(AVG_PAY, Treatment(reference=0))[T.[2217.08, 10116.92)]     -0.3201   0.0549  -5.8319 0.0000 -0.4277 -0.2125
==============================================================================================================

```